In [1]:
# Initial imports.
import pandas as pd
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,balanced_accuracy_score,r2_score,mean_squared_error

In [2]:
global_data_df = pd.read_csv('global_health_data.csv')
global_data_df["both_sex_life_exp"]=round(global_data_df["both_sex_life_exp"],3) # change this back to 3 decimal places
global_data_df

,country_code,country,co,no,no2,so2,pm2_5,pm10,gdp_per_capita,Prevalence of BMI>=30 kg/m≤ (obesity),Prevalence of BMI<18.5 kg/m≤ (underweight),che_per_capita_usd,both_sex_life_exp,fem_life_exp,male_life_exp,poverty_rate
0,AF,Afghanistan,1350.724521,0.683179,12.363946,0.759946,17.227616,42.560698,2057.0,0.022760,0.229979,6680.0,62.430,62.755,62.165,54.5
1,AL,Albania,257.109347,0.362959,6.039690,1.989154,10.260962,14.133179,12783.0,0.129874,0.021329,1867.0,77.920,79.800,76.175,14.3
2,DZ,Algeria,241.442119,3.336000,13.501865,2.798434,12.425665,37.320633,11683.0,0.162522,0.066905,2646.0,76.835,77.810,75.940,5.5
3,AO,Angola,471.672875,0.718363,5.120278,3.372409,18.544950,29.713428,7304.0,0.034766,0.178649,3878.0,62.390,65.010,59.915,32.3
4,AR,Argentina,423.765962,8.538443,11.451818,5.575630,7.109720,9.720282,23107.0,0.197364,0.015318,2981.0,76.375,79.400,73.215,35.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,UZ,Uzbekistan,273.726837,4.314358,12.812164,1.456380,8.780014,19.103754,6867.0,0.095002,0.047757,6001.0,72.230,74.565,69.870,14.1
145,VU,Vanuatu,199.313946,0.010838,0.226337,0.407213,2.023984,4.395204,3040.0,0.147728,0.036530,3745.0,64.975,68.000,62.340,12.7
146,VN,Vietnam,869.873588,6.159495,18.272751,15.109423,43.126799,52.487995,7198.0,0.007037,0.248009,1190.0,73.450,77.860,69.220,6.7
147,ZM,Zambia,526.302397,0.261384,1.657479,5.618650,7.969808,11.588515,3478.0,0.042148,0.142703,5094.0,61.475,64.195,58.735,54.4


In [3]:
mean=global_data_df["both_sex_life_exp"].mean()
stdev=global_data_df["both_sex_life_exp"].std()
print(mean)
print(stdev)
print(f"Use {round(mean+stdev,1)} years as the threshold for the high_life_exp group")
print(f"Use {round(mean-stdev,1)} years as the threshold for the low_life_exp group")

71.87003355704702
7.534038759699217
Use 79.4 years as the threshold for the high_life_exp group
Use 64.3 years as the threshold for the low_life_exp group


In [4]:
# create a new column called life_expenctancy_group
bins = [0, 64.3, 79.4, 100]
labels = ['low', 'medium', 'high']
global_data_df['life_exp_group'] = pd.cut(global_data_df['both_sex_life_exp'], bins=bins, labels=labels)
global_data_df['life_exp_encoded'] = pd.cut(global_data_df['both_sex_life_exp'], bins=bins, labels=[0, 1, 2], include_lowest=True)
global_data_df

,country_code,country,co,no,no2,so2,pm2_5,pm10,gdp_per_capita,Prevalence of BMI>=30 kg/m≤ (obesity),Prevalence of BMI<18.5 kg/m≤ (underweight),che_per_capita_usd,both_sex_life_exp,fem_life_exp,male_life_exp,poverty_rate,life_exp_group,life_exp_encoded
0,AF,Afghanistan,1350.724521,0.683179,12.363946,0.759946,17.227616,42.560698,2057.0,0.022760,0.229979,6680.0,62.430,62.755,62.165,54.5,low,0
1,AL,Albania,257.109347,0.362959,6.039690,1.989154,10.260962,14.133179,12783.0,0.129874,0.021329,1867.0,77.920,79.800,76.175,14.3,medium,1
2,DZ,Algeria,241.442119,3.336000,13.501865,2.798434,12.425665,37.320633,11683.0,0.162522,0.066905,2646.0,76.835,77.810,75.940,5.5,medium,1
3,AO,Angola,471.672875,0.718363,5.120278,3.372409,18.544950,29.713428,7304.0,0.034766,0.178649,3878.0,62.390,65.010,59.915,32.3,low,0
4,AR,Argentina,423.765962,8.538443,11.451818,5.575630,7.109720,9.720282,23107.0,0.197364,0.015318,2981.0,76.375,79.400,73.215,35.5,medium,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,UZ,Uzbekistan,273.726837,4.314358,12.812164,1.456380,8.780014,19.103754,6867.0,0.095002,0.047757,6001.0,72.230,74.565,69.870,14.1,medium,1
145,VU,Vanuatu,199.313946,0.010838,0.226337,0.407213,2.023984,4.395204,3040.0,0.147728,0.036530,3745.0,64.975,68.000,62.340,12.7,medium,1
146,VN,Vietnam,869.873588,6.159495,18.272751,15.109423,43.126799,52.487995,7198.0,0.007037,0.248009,1190.0,73.450,77.860,69.220,6.7,medium,1
147,ZM,Zambia,526.302397,0.261384,1.657479,5.618650,7.969808,11.588515,3478.0,0.042148,0.142703,5094.0,61.475,64.195,58.735,54.4,low,0


In [5]:
# drop the country_code and country columns from the data set
global_data=global_data_df.drop(columns=["country_code","country","fem_life_exp","male_life_exp"])

In [6]:
global_data = global_data.dropna()
global_data

,co,no,no2,so2,pm2_5,pm10,gdp_per_capita,Prevalence of BMI>=30 kg/m≤ (obesity),Prevalence of BMI<18.5 kg/m≤ (underweight),che_per_capita_usd,both_sex_life_exp,poverty_rate,life_exp_group,life_exp_encoded
0,1350.724521,0.683179,12.363946,0.759946,17.227616,42.560698,2057.0,0.022760,0.229979,6680.0,62.430,54.5,low,0
1,257.109347,0.362959,6.039690,1.989154,10.260962,14.133179,12783.0,0.129874,0.021329,1867.0,77.920,14.3,medium,1
2,241.442119,3.336000,13.501865,2.798434,12.425665,37.320633,11683.0,0.162522,0.066905,2646.0,76.835,5.5,medium,1
3,471.672875,0.718363,5.120278,3.372409,18.544950,29.713428,7304.0,0.034766,0.178649,3878.0,62.390,32.3,low,0
4,423.765962,8.538443,11.451818,5.575630,7.109720,9.720282,23107.0,0.197364,0.015318,2981.0,76.375,35.5,medium,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,273.726837,4.314358,12.812164,1.456380,8.780014,19.103754,6867.0,0.095002,0.047757,6001.0,72.230,14.1,medium,1
145,199.313946,0.010838,0.226337,0.407213,2.023984,4.395204,3040.0,0.147728,0.036530,3745.0,64.975,12.7,medium,1
146,869.873588,6.159495,18.272751,15.109423,43.126799,52.487995,7198.0,0.007037,0.248009,1190.0,73.450,6.7,medium,1
147,526.302397,0.261384,1.657479,5.618650,7.969808,11.588515,3478.0,0.042148,0.142703,5094.0,61.475,54.4,low,0


In [7]:
X = global_data.drop(columns=["both_sex_life_exp","life_exp_group","life_exp_encoded"])
X

,co,no,no2,so2,pm2_5,pm10,gdp_per_capita,Prevalence of BMI>=30 kg/m≤ (obesity),Prevalence of BMI<18.5 kg/m≤ (underweight),che_per_capita_usd,poverty_rate
0,1350.724521,0.683179,12.363946,0.759946,17.227616,42.560698,2057.0,0.022760,0.229979,6680.0,54.5
1,257.109347,0.362959,6.039690,1.989154,10.260962,14.133179,12783.0,0.129874,0.021329,1867.0,14.3
2,241.442119,3.336000,13.501865,2.798434,12.425665,37.320633,11683.0,0.162522,0.066905,2646.0,5.5
3,471.672875,0.718363,5.120278,3.372409,18.544950,29.713428,7304.0,0.034766,0.178649,3878.0,32.3
4,423.765962,8.538443,11.451818,5.575630,7.109720,9.720282,23107.0,0.197364,0.015318,2981.0,35.5
...,...,...,...,...,...,...,...,...,...,...,...
144,273.726837,4.314358,12.812164,1.456380,8.780014,19.103754,6867.0,0.095002,0.047757,6001.0,14.1
145,199.313946,0.010838,0.226337,0.407213,2.023984,4.395204,3040.0,0.147728,0.036530,3745.0,12.7
146,869.873588,6.159495,18.272751,15.109423,43.126799,52.487995,7198.0,0.007037,0.248009,1190.0,6.7
147,526.302397,0.261384,1.657479,5.618650,7.969808,11.588515,3478.0,0.042148,0.142703,5094.0,54.4


In [8]:
y = global_data["life_exp_encoded"]
y_1 = global_data["both_sex_life_exp"]

In [9]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [10]:
# Try to do StandardScaler later in this same cell.
# Creating a StandardScaler instance.
scaler = StandardScaler() # use this for StandardScaler scaling
#minmaxscaler = MinMaxScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train) # standardscaler
# X_minmaxscaler = minmaxscaler.fit(X_train)
# Scaling the data.
## For StandardScaler scaling, sub X_minmaxscaler for X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Instantiate a LogisticRegression model
model = LogisticRegression(solver = 'lbfgs',random_state=1)

In [12]:
# Fitting the model
#rf_model = rf_model.fit(X_train_scaled, y_train)
model = model.fit(X_train_scaled, y_train)

In [13]:
# Making predictions using the testing data.
y_pred = model.predict(X_test_scaled)

In [14]:
# Print the accuracy score for the training and testing data for the LogisticRegression ML model
print(model.score(X_train_scaled, y_train))
print(model.score(X_test_scaled, y_test))

0.8558558558558559
0.868421052631579


In [15]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.9358974358974358

In [16]:
cm = confusion_matrix(y_test,y_pred)
cm

array([[ 7,  0,  0],
       [ 5, 21,  0],
       [ 0,  0,  5]])

In [17]:
# Classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.58      1.00      0.74         7
           1       1.00      0.81      0.89        26
           2       1.00      1.00      1.00         5

    accuracy                           0.87        38
   macro avg       0.86      0.94      0.88        38
weighted avg       0.92      0.87      0.88        38



In [18]:
# Categorical predictions
y_test_df = pd.DataFrame(y_test)
y_test_df["predicted_values"]=y_pred
y_test_df=y_test_df.sort_index()

In [19]:
# isolate countries with predictions:
predicted_countries = global_data_df[["country_code","country","life_exp_group"]]


In [20]:
# merge the predicted_df with the original dataset using the index values
categorical_life_exp_predictions = predicted_countries.join(y_test_df, how='inner')
categorical_life_exp_predictions

,country_code,country,life_exp_group,life_exp_encoded,predicted_values
2,DZ,Algeria,medium,1,1
4,AR,Argentina,medium,1,1
7,AT,Austria,high,2,2
15,BA,Bosnia and Herzegovina,medium,1,1
19,BF,Burkina Faso,low,0,0
21,KH,Cambodia,medium,1,1
30,CG,Congo,low,0,0
33,HR,Croatia,medium,1,1
34,CY,Cyprus,high,2,2
42,SV,El Salvador,medium,1,1


In [21]:
"""best_r2_score = 0
for i in range(50,100):
    X_train, X_test, y_train, y_test = train_test_split(X, y_1, random_state=99)
    rf_model = RandomForestRegressor(n_estimators = 1024, random_state = 99)
    rf_model = rf_model.fit(X_train_scaled, y_train)
    y_pred = rf_model.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred)
    if r2 > best_r2_score:
        best_r2_score = r2
        best_condition = i
print(f"The best r2 score is {best_r2_score} and comes from random_state={i}")
"""

'best_r2_score = 0\nfor i in range(50,100):\n    X_train, X_test, y_train, y_test = train_test_split(X, y_1, random_state=99)\n    rf_model = RandomForestRegressor(n_estimators = 1024, random_state = 99)\n    rf_model = rf_model.fit(X_train_scaled, y_train)\n    y_pred = rf_model.predict(X_test_scaled)\n    r2 = r2_score(y_test, y_pred)\n    if r2 > best_r2_score:\n        best_r2_score = r2\n        best_condition = i\nprint(f"The best r2 score is {best_r2_score} and comes from random_state={i}")\n'

In [22]:
# Predicting Continous Data
# Splitting into Train and Test sets. Specifically uses y_1 as the original values for life expectancy (continuous)
X_train, X_test, y_train, y_test = train_test_split(X, y_1, random_state=78)

In [23]:
# Try to do StandardScaler later in this same cell.
# Creating a StandardScaler instance.
scaler = StandardScaler() # use this for StandardScaler scaling
#minmaxscaler = MinMaxScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train) # standardscaler
# X_minmaxscaler = minmaxscaler.fit(X_train)
# Scaling the data.
## For StandardScaler scaling, sub X_minmaxscaler for X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:
# Create a random forest regressor. Change lasso back to RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators = 100, random_state = 78)
rf_model = rf_model.fit(X_train_scaled, y_train)
y_pred = rf_model.predict(X_test_scaled)

In [25]:
# Print the accuracy score for training and testing data for the RandomForestRegressor
print(rf_model.score(X_train_scaled, y_train))
print(rf_model.score(X_test_scaled, y_test))

0.9657194016908732
0.8273245045225258


In [26]:
mse = round(mean_squared_error(y_test,y_pred),2)
print(f"The mean squared error is: {mse}") 

The mean squared error is: 8.42


In [27]:
# Test the accuracy of the RandomForestRegressor model with R^2
r2 = r2_score(y_test, y_pred)
r2

0.8273245045225258

In [28]:
# Continuous Life expectancy (floats) predictions
y_test_df = pd.DataFrame(y_test)
y_test_df["predicted_values"]=y_pred
y_test_df=y_test_df.sort_index()

In [29]:
# merge the predicted_df with the original dataset using the index values
continuous_life_exp_predictions = predicted_countries.join(y_test_df, how='inner')
continuous_life_exp_predictions

,country_code,country,life_exp_group,both_sex_life_exp,predicted_values
2,DZ,Algeria,medium,76.835,74.86170
4,AR,Argentina,medium,76.375,75.65500
7,AT,Austria,high,81.315,81.74470
15,BA,Bosnia and Herzegovina,medium,76.535,73.72860
19,BF,Burkina Faso,low,61.810,65.15885
21,KH,Cambodia,medium,69.645,69.74550
30,CG,Congo,low,63.925,62.40975
33,HR,Croatia,medium,78.110,77.32785
34,CY,Cyprus,high,82.575,80.48075
42,SV,El Salvador,medium,74.140,73.67950


In [30]:
# Next step is look for correlations in the data
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
import scipy
from sklearn.linear_model import LinearRegression
linearregression_model = LinearRegression()
linearregression_model.fit(X_train_scaled, y_train)

# Use our model to predict a value
predicted = linearregression_model.predict(X_test_scaled)

mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

print(f"mean squared error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")




mean squared error (MSE): 17.787263795228103
R-squared (R2 ): 0.635355641272865


In [31]:
coeffs = linearregression_model.coef_
coeffs
corrs = np.corrcoef(X_train.T, y_train)
pearson_coeffs = corrs[:-1, -1]
pearson_coeffs

array([-0.05283057,  0.18122943,  0.36272092,  0.15484844, -0.22040222,
       -0.31060901,  0.70956677,  0.45011965, -0.60367533,  0.17098861,
       -0.69363278])

In [32]:
coeffs = [x for x in pearson_coeffs]
correlations_data = global_data.drop(columns=["both_sex_life_exp","life_exp_group","life_exp_encoded"])
correlations_data
cols = [x for x in correlations_data.columns]
data = {'features': cols, 'r_values': coeffs}
correlations_df=pd.DataFrame(data)
correlations_df

,features,r_values
0,co,-0.052831
1,no,0.181229
2,no2,0.362721
3,so2,0.154848
4,pm2_5,-0.220402
5,pm10,-0.310609
6,gdp_per_capita,0.709567
7,Prevalence of BMI>=30 kg/m≤ (obesity),0.450120
8,Prevalence of BMI<18.5 kg/m≤ (underweight),-0.603675
9,che_per_capita_usd,0.170989


In [33]:
for i,row in correlations_df.iterrows():
    feature=row["features"]
    r = row["r_values"]
    n=38
    df=n-2
    t = r * math.sqrt(df / (1 - r**2))
    p_value = 1 - scipy.stats.t.cdf(t, df)
    #row["p_value"]=p_value
    correlations_df.at[i, "p_value"] = p_value
    print(f"The p-value for {feature} feature correlation with life expectancy is: {p_value}")

The p-value for co feature correlation with life expectancy is: 0.6236231818228004
The p-value for no feature correlation with life expectancy is: 0.13809859467124808
The p-value for no2 feature correlation with life expectancy is: 0.012607169622736891
The p-value for so2 feature correlation with life expectancy is: 0.1766314543962484
The p-value for pm2_5 feature correlation with life expectancy is: 0.9081888368226331
The p-value for pm10 feature correlation with life expectancy is: 0.9711524115325098
The p-value for gdp_per_capita feature correlation with life expectancy is: 3.054086451514948e-07
The p-value for Prevalence of BMI>=30 kg/m≤ (obesity) feature correlation with life expectancy is: 0.002287231270265533
The p-value for Prevalence of BMI<18.5 kg/m≤ (underweight) feature correlation with life expectancy is: 0.9999699510447388
The p-value for che_per_capita_usd feature correlation with life expectancy is: 0.15234733392474986
The p-value for poverty_rate feature correlation wi

In [34]:
correlations_df=correlations_df.sort_values("p_value",ascending=True)
correlations_df

,features,r_values,p_value
6,gdp_per_capita,0.709567,3.054086e-07
7,Prevalence of BMI>=30 kg/m≤ (obesity),0.450120,2.287231e-03
2,no2,0.362721,1.260717e-02
1,no,0.181229,1.380986e-01
9,che_per_capita_usd,0.170989,1.523473e-01
3,so2,0.154848,1.766315e-01
0,co,-0.052831,6.236232e-01
4,pm2_5,-0.220402,9.081888e-01
5,pm10,-0.310609,9.711524e-01
8,Prevalence of BMI<18.5 kg/m≤ (underweight),-0.603675,9.999700e-01
